# 0. Install and Import Dependencies

In [ ]:
# !pip install tensorflow-hub opencv-python matplotlib pandas shapely

In [2]:
import os,math
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import pprint
import random
import court.utils as utils 
import court.court as court


In [3]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 1. Load Model

In [4]:
model = hub.load("https://tfhub.dev/google/movenet/multipose/lightning/1")
movenet = model.signatures['serving_default']


Metal device set to: Apple M2 Pro


# 2. Draw People

In [5]:
def draw_keypoints(frame, shaped, confidence_threshold, color):
    for kp in shaped:
        ky, kx = kp
        cv2.circle(frame, (int(kx), int(ky)), 4, color, -1)

In [6]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [7]:
def draw_connections(frame, shaped, edges, confidence_threshold):
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1 = shaped[p1]
        y2, x2 = shaped[p2]
        cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,int(y2)), 4)

In [8]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    i=126
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold, (0,i,0))
        i=255
    

# 3. Make Detections

In [9]:

FILE_RES="/Users/sunny/Desktop/ai_badminton/part1/train/"  # 原影片的資料夾
FILE_DST="/Users/sunny/move_dataset/" # 要存 move_data 資料夾
CONFIDENCE_THRESHOLD = 0.1 # 判斷點的信心分數大於 CONFIDENCE_THRESHOLD
MIN_CONFIDENCE_NUM = 0 # 17個點至少要有 MIN_CONFIDENCE_NUM 的點大於 CONFIDENCE_THRESHOLD 才會拿 movnet預測的資料，否則拿前一幀的資料
SAVE_DATA = False # 要不要存檔案


In [13]:
# detection and save A B all move data
#744
for num in tqdm(range(1,801)):
    
    print(num)
    
    file_name = '%05d' % num
    cap = cv2.VideoCapture(f'{FILE_RES}{file_name}/{file_name}.mp4')
    ret, frame = cap.read()
    cap.release()
    
        
    a,c = court.get_court(frame)
    corner=[]
    for i in range(0,4):
        corner.append((int(a[i][0]),int(a[i][1]+20)))
    for i in range(20,24):
        corner.append((int(a[i][0]),int(a[i][1]-20)))   
    center = (int((c[0][0]+c[1][0])/2), int((c[1][1]+c[1][1])/2))

        
    hitframe = 1
    index = 0

    y, x = 720, 1280
    A_player = []
    B_player = []
    A_label = []
    B_label = []
    
    cap = cv2.VideoCapture(f'{FILE_RES}{file_name}/{file_name}.mp4')

    while True:
        ret, frame = cap.read()

        if frame is None:
            break

        # Resize image
        img = frame.copy()
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 352,640)
        input_img = tf.cast(img, dtype=tf.int32)


        # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))

        
        # Find the players
        for person in keypoints_with_scores:
            # y x
            shaped = np.squeeze(np.multiply(person, [y,x,1]))
            point = Point(shaped[16][1], shaped[16][0])
            polygon = Polygon(corner)

            if polygon.contains(point):

                if point.y > center[1] and (np.count_nonzero(shaped[:,2]>= CONFIDENCE_THRESHOLD)) > MIN_CONFIDENCE_NUM:
                    B_prev = shaped

                elif point.y <= center[1] and (np.count_nonzero(shaped[:,2] >= CONFIDENCE_THRESHOLD)) > MIN_CONFIDENCE_NUM:
                    A_prev = shaped

        A_player.append(A_prev)
        B_player.append(B_prev)
        
        

            
    
        # Show the image
        
        # loop_through_people(frame, target, EDGES, 0.1)
        # print("draw")
        
        #A
        draw_connections(frame, A_prev[:,:2], EDGES, 0.1)
        draw_keypoints(frame, A_prev[:,:2], 0.1, (0,128,0))
        #B
        draw_connections(frame, B_prev[:,:2], EDGES, 0.1)
        draw_keypoints(frame, B_prev[:,:2], 0.1, (0,255,0))
        
        cv2.circle(frame , center, 5 , (0, 255, 255),-1)
        cv2.circle(frame , corner[0], 5 , (0, 255, 255),-1)
        cv2.circle(frame , corner[1], 5 , (0, 255, 255),-1)
        cv2.circle(frame , corner[2], 5 , (0, 255, 255),-1)
        cv2.circle(frame , corner[3], 5 , (0, 255, 255),-1)
        cv2.circle(frame , corner[4], 5 , (0, 255, 255),-1)
        cv2.circle(frame , corner[5], 5 , (0, 255, 255),-1)
        cv2.circle(frame , corner[6], 5 , (0, 255, 255),-1)
        cv2.circle(frame , corner[7], 5 , (0, 255, 255),-1)
        cv2.imshow('Movenet Multipose', frame)  
        

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
            
        hitframe += 1
        # break
        
    
    # Save data
    if SAVE_DATA:
        np.save(f'{FILE_DST}{file_name}_A_move', A_player)
        np.save(f'{FILE_DST}{file_name}_B_move', B_player)
        # np.save(f'{FILE_DST}{file_name}_A_hit_label', A_label)
        # np.save(f'{FILE_DST}{file_name}_B_hit_label', B_label)
    
    # print(hitframe)
cap.release()
cv2.waitKey()
cv2.destroyAllWindows()

  0%|          | 0/800 [00:00<?, ?it/s]

1
2
3


KeyboardInterrupt: 

In [ ]:
# show A B all move data

for num in tqdm(range(1,801)):
    
    file_name = '%05d' % num
    
    if not os.path.exists(f'{FILE_DST}{file_name}_A_move.npy'):
        continue
    
    cap = cv2.VideoCapture(f'{FILE_RES}{file_name}/{file_name}.mp4')
    A_player = np.load(f'{FILE_DST}{file_name}_A_move.npy')
    B_player = np.load(f'{FILE_DST}{file_name}_B_move.npy')
        
    hitframe = 0
    index = 0

    while True:
        ret, frame = cap.read()


        if frame is None:
            break

    
        # Show the image
        
        # A
        draw_connections(frame, A_player[hitframe][:,:2], EDGES, 0.1)
        draw_keypoints(frame, A_player[hitframe][:,:2], 0.1, (0,128,0))
        
        # B
        draw_connections(frame, B_player[hitframe][:,:2], EDGES, 0.1)
        draw_keypoints(frame, B_player[hitframe][:,:2], 0.1, (0,255,0))
       
        cv2.imshow('Movenet Multipose', frame)        
        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
            
        hitframe += 1
        
    # print(hitframe)
    
cap.release()    
cv2.waitKey()
cv2.destroyAllWindows()


In [ ]:
# test
# train_A = np.load(f'/Users/sunny/move_dataset/{file_name}_A_move.npy')
# train_B = np.load(f'/Users/sunny/move_dataset/{file_name}_B_move.npy')
# label_A = np.load(f'/Users/sunny/move_dataset/{file_name}_A_hit_label.npy')
# label_B = np.load(f'/Users/sunny/move_dataset/{file_name}_B_hit_label.npy')
# print(label_A[:40])
# print(label_B[:40])

# print((train_A == A_player).all())
# print((train_B == B_player).all())
# print((label_A == A_label).all())
# print((label_B == B_label).all())

# print(np.array(A_player).shape)
# print(np.array(B_player).shape)

# print(A_label[100:140])
# print(B_label[100:140])

# Homography
# pts_src = np.array([[141, 131], [480, 159], [493, 630],[64, 601]])
# pts_dst = np.array([[0, 0],[256, 0],[256, 256],[0, 256]])
# h, status = cv2.findHomography(pts_src, pts_dst)
# im_out = cv2.warpPerspective(im_src, h, (im_dst.shape[1],im_dst.shape[0]))

# load data

# for num in tqdm(range(1,801)):
#     # print(num)
#     file_name = '%05d' % num
#     if not os.path.exists(f'{FILE_DST}{file_name}{file_name}_A_move.npy'):
#         continue
        
#     A_player = np.load(f'{FILE_DST}{file_name}_A_move.npy')
#     B_player = np.load(f'{FILE_DST}{file_name}_B_move.npy')
    
